<a href="https://colab.research.google.com/github/GNunes-br/nlp_sentiment_analysis_pt3/blob/main/Machine_Learning_para_classificar_textos_Parte_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 10.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
from nltk import tokenize

import seaborn as sns

from string import punctuation

import unidecode

In [ ]:
driveSource = '/content/drive/MyDrive/Colab Notebooks/datasets'

In [ ]:
dataset = pd.read_csv(driveSource + "/imdb_pt_br.csv")

In [ ]:
dataset

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos


In [ ]:
classification = dataset.sentiment.replace(['neg', 'pos'], [0,1])
dataset["classif"] = classification

In [ ]:
nltk.download('all', quiet=True)

True

In [ ]:
def getStopWordsList():

  return nltk.corpus.stopwords.words('portuguese')  

In [ ]:
def getPunctuationList():
  punctuationList = list()

  for value in punctuation:
    punctuationList.append(value)

  return punctuationList

In [ ]:
stopWords = [unidecode.unidecode(text) for text in (getStopWordsList() + getPunctuationList())]

In [ ]:
wordPunctTokenizer = tokenize.WordPunctTokenizer()

In [ ]:
whitespaceTokenizer = tokenize.WhitespaceTokenizer()

In [ ]:
stemmer = nltk.RSLPStemmer()

In [ ]:
def processPhrase(text):
  newPhrase = list()

  convertTextToLowercase = text.lower()

  removeTextAccents = unidecode.unidecode(convertTextToLowercase)

  textWordPunctTokenizer = wordPunctTokenizer.tokenize(removeTextAccents)

  textWhiteSpaceTokenizer = whitespaceTokenizer.tokenize(' '.join(textWordPunctTokenizer))

  textWords = textWhiteSpaceTokenizer

  for word in textWords:
    if word not in stopWords:
      newPhrase.append(stemmer.stem(word))

  return ' '.join(newPhrase)

In [ ]:
processedText = list();

for text in dataset.text_pt:
  
  processedText.append(processPhrase(text))

In [ ]:
dataset['processed'] = processedText

In [ ]:
tfidf = TfidfVectorizer(lowercase=False, ngram_range=(1,2))

In [ ]:
dataset.head()

,id,text_en,text_pt,sentiment,classif,processed
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0,vez sr costn arrum film temp necessari alem te...
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,0,exempl motiv maior film aca mesm gener chat na...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0,prim tud odei rap imbecil pod agir arm pressio...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,0,beatl pud escrev music tod gost emb walt hill ...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,0,film fot lata palavr apropri verdad tant ous q...


In [ ]:
tfidfVector = tfidf.fit_transform(dataset.processed)

train, test, trainClass, testClass = train_test_split(tfidfVector, dataset["classif"], random_state=42)

In [ ]:
logisticRegression = LogisticRegression()

In [ ]:
logisticRegression.fit(train, trainClass)

LogisticRegression()

In [ ]:
acuracy = logisticRegression.score(test, testClass)

In [ ]:
acuracy

0.8856449656287909

In [ ]:
testPhrases = [
    processPhrase('Gostei muito do filme, me emocionei durante as cenas, o resultado ficou show!'),
    processPhrase('Não gostei muito do filme, não me emocionei durante as cenas, o resultado ficou ruim!'),
    processPhrase('Amei o filme!'),
]

In [ ]:
testPhrases

['gost film emocion dur cen result fic show',
 'gost film emocion dur cen result fic ruim',
 'ame film']

In [ ]:
logisticRegression.predict(tfidf.transform(testPhrases))

array([1, 0, 1])